This notebook was created by Donna Faith Go.

In [4]:
import yfinance as yf
import pandas as pd 
import pickle

# GARCH on the PSEi

This notebook aims to do volatility clustering on the Philippine Stock Exchange Composite Index (PSEi) through the Generalized Autoregressive Conditional Heteroskedasticity (GARCH) model.

## Data Gathering

In [5]:
# getting closing prices for the 30 stocks with batching
start_date = '2013-01-01'# '2019-01-01'
end_date = '2026-01-01' #'2025-08-01'

def download_stocks_in_batches(tickers, batch_size=5, delay=1):
    """
    Download stock data in batches to avoid rate limiting
    """
    all_data = {}
    
    for i in range(0, len(tickers), batch_size):
        batch = tickers[i:i + batch_size]
        print(f"Downloading batch {i//batch_size + 1}: {batch}")
        
        try:
            # Download the batch
            batch_data = yf.download(
                batch,
                start=start_date,
                end=end_date,
                progress=False
            )
            
            # Extract closing prices for this batch
            if not batch_data.empty and 'Close' in batch_data.columns:
                closes = batch_data['Close']
                if isinstance(closes, pd.Series):
                    all_data[batch[0]] = closes
                else:
                    for ticker in closes.columns:
                        all_data[ticker] = closes[ticker]
                print(f"Successfully downloaded {len(batch)} stocks")
            else:
                print(f"No data returned for batch: {batch}")
            
        except Exception as e:
            print(f"Error downloading batch {batch}: {e}")
        
        # Add delay to avoid rate limiting
        if i + batch_size < len(tickers):
            print(f"Waiting {delay} seconds before next batch...")
            time.sleep(delay)
    
    if all_data:
        return pd.DataFrame(all_data)
    else:
        return pd.DataFrame()

# Download in batches of 5 stocks with 1-second delay
closing_df = download_stocks_in_batches(
    ['PSEI.PS'], 
    batch_size=5, 
    delay=5
)

if not closing_df.empty:
    closing_df.to_pickle('psei closing prices.pkl')

C:\Users\Donna\AppData\Local\Temp\ipykernel_26432\1752588761.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  batch_data = yf.download(


Successfully downloaded 1 stocks


In [ ]:

filepath = r'psei prices.pkl'
with open(filepath, 'rb') as f:
    data = pickle.load(f)
data.tail()